# A la maniere de 'continuum' (Gyorgi Ligeti)

This notebook generates a version of Ligeti's Continuum texture using stocastic processes controlled by line envelopes.

<hr style="height:1px;color:gray">

Imports from the musx package, see [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for more information:

In [ ]:
from musx import Score, Note, Seq, MidiFile, Shuffle, Choose, scale, setmidiplayer, playfile
from musx.midi.gm import Harpsichord
print('OK!')

If you have installed a terminal-based midi player you can autoplay this notebook's generated midi file(s) by deleting the *#* comment sign below and providing your  terminal play command without including any file name. The unmodified example would call [fluidsynth](https://www.fluidsynth.org/download/) and pass it an instance of the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) soundfont stored in /usr/local/sf/:

In [ ]:
# setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")

The Continuum generator calls the register() generator to create successive sections of the composition:

In [ ]:
def continuum (score, rhy, minkeys, maxkeys, seclens):
    """
    Calls register() to create the next section's material, then
    waits until that section is over before creating another section.

    Parameters
    ----------
    score : Score
        The scheduling queue to run the composer in.
    rhy : int | float
        The amount of time to wait between this note and the next.
    minkeys : list
        A list of keynums defining the lowest possible note for each call to register().
    maxkeys : list
        A list of keynums defining the highest possible note for each call to register().
    seclens : list
        A list of section lengths, in seconds, for the entire composition.

    """
    # random pattern of section lengths.
    pat = Choose(seclens)
    # iterate all the min and max key numbers 
    for low, high in zip(minkeys, maxkeys):
        # get the section's duration
        secdur = pat.next()
        # sprout the next section
        score.compose(register(score, rhy, secdur, low, high, .4))
        # wait till end of section
        yield secdur
        
print(f"continuum: {continuum}")

The register generator composes the notes for a section of music:

In [ ]:
def register (score, rhy, dur, low, high, amp):
    """
    Creates a chromatic scale between low and high, notes in the scale
    are shuffled to procduce a saturated texture.
    """ 
    pat = Shuffle(scale(low, high-low+1, 1))
    while score.elapsed < dur:
        keyn = pat.next()
        note = Note(time=score.now, duration=rhy, pitch=keyn, amplitude=amp)
        score.add(note)
        yield rhy

print(f"register: {register}")

Define track0 to be a midi meta track that holds tempo, midi instrument assignments, micro tuning, etc.:

In [ ]:
track0 = MidiFile.metatrack(ins={0: Harpsichord})
print(f"track0: {track0}")

Track1 will hold the composition:

In [ ]:
track1 = Seq()
print(f"track1:", track1)

Create a score and pass it track1 to hold the output midi event data:

In [ ]:
score = Score(out=track1)
print(f"score: {score}")

Lower and upper bounds on keynum choices:

In [ ]:
minkeys = [60, 59, 58, 57, 56, 55, 54, 53, 52, 
           53, 54, 55, 56, 57, 58, 59, 60, 61, 
           62, 63, 64, 65, 66, 67, 68, 69, 70,
           71, 72, 73, 74, 75, 76, 77, 78, 79, 
           80, 82, 83, 84, 85, 86, 87, 88, 89, 89]
print(f"minkeys: {minkeys}")

maxkeys = [62, 63, 64, 65, 66, 67, 68, 69, 70, 
           70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70,
           71, 72, 73, 74, 76, 79, 83, 86, 88, 89,
           89, 89, 89, 89, 89, 89, 89, 89, 89, 89, 89]
print(f"maxkeys: {maxkeys}")

Length of sections:

In [ ]:
seclens = [.5, 1, 1.5, 2, 2.5]
print(f"seclens: {seclens}")

Speed of rhythm:

In [ ]:
rate = .075
print(f"rate: {rate}")

Create the composition:

In [ ]:
score.compose(continuum(score, rate, minkeys, maxkeys, seclens))
print(f"score: {score}")

Write the tracks to a midi file in the current directory:

In [ ]:
file = MidiFile("continuum.mid", [track0, track1]).write()
print(f"Wrote '{file.pathname}'")

Play the output midi file if you activated setmidiplayer() at the top of the file:

In [ ]:
playfile(file.pathname)